In [5]:
import polars as pl
from spatial_filtering import arrays, constants, direction_of_arrival
import importlib

importlib.reload(arrays)
importlib.reload(constants)
importlib.reload(direction_of_arrival)
data = pl.read_excel("meerkat positions.xlsx")


In [6]:
data.head()

Antenna,East,North,Up
str,f64,f64,f64
"""m000""",-8.264,-207.29,8.597
"""m001""",1.121,-171.762,8.471
"""m002""",-32.113,-224.236,8.645
"""m003""",-66.518,-202.276,8.285
"""m004""",-123.624,-252.946,8.513


In [7]:
data_np = data[['East', 'North', 'Up']].to_numpy()

In [ ]:
from spatial_filtering.arrays import Array
import numpy as np

array = arrays.Array(data_np)
N = array.num_antennas
f = 1.6e9              # Frequency 1.6 GHz
wavelength = constants.c / f

r_source = [200, 20, 4000, 2500, 3000]        # meters
theta_source_deg = [20, 75, 40, 88, 5]
theta_source = np.deg2rad(theta_source_deg)

r_steps = 5001
theta_steps = 91 * 2 ** 10

# Signal parameters
SNR_dB = 40
SNR = 10**(SNR_dB / 10)
snapshots = 2000


X = np.zeros((N, snapshots), dtype=np.complex128)
# Generate signal
for (r, theta) in zip(r_source, theta_source):
    a = array.nf_steering_vector(r, [0, theta], wavelength).reshape(-1,1)
    signal = (np.random.randn(1, snapshots) + 1j * np.random.randn(1, snapshots)) / np.sqrt(2)  # complex Gaussian signal
    X += a @ signal
noise_power = 1 / SNR
noise = np.sqrt(noise_power / 2) * (np.random.randn(N, snapshots) + 1j * np.random.randn(N, snapshots))

X += noise

# Received signal
# Covariance matrix
R = X @ X.conj().T / N 

output = direction_of_arrival.MUSICNF2D().get_direction(
    array,
    R, 
    num_interferers=len(r_source),
    wavelength = wavelength,
    r_min=0,
    r_max = 5000,
    r_steps=r_steps,
    theta_min_deg = 0,
    theta_max_deg = 90,
    theta_steps=theta_steps,
    phi_min_deg = 0,
    phi_max_deg = 0,
    phi_steps=1,
)

In [ ]:

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'  # or 'iframe' or 'colab' depending on your environment

# Filter top 1% Q values (adjust as needed)
threshold = output['Q'].quantile(0.999975)
high_Q = output.filter(pl.col('Q') >= threshold)

# Create 3D scatter plot
fig = px.scatter_3d(
    high_Q,
    x='theta',  # azimuth
    y='r',      # range
    z='Q',      # MUSIC power
    color='Q',
    color_continuous_scale='Jet',
    opacity=0.8
)

# Update layout for better visuals
fig.update_layout(
    title='High-Q MUSIC Spectrum Peaks',
    scene=dict(
        xaxis_title='Angle (degrees)',
        yaxis_title='Range (meters)',
        zaxis_title='Q (dB)'
    )
)

fig.show()

In [ ]:
for r, theta in zip(r_source, theta_source_deg):
    print(f"{r} / {theta} deg")